In [142]:
import json
import os
import sys
import quantstats
import plotly.express as px

from ISIN import ISIN, FUND_NAME

FUND_info_path = "raw_data/Fund_info"
Daily_NAV_path = "raw_data/Daily_NAV"


In [143]:
def invert_dict(dictionary: dict):
    return {v: k for k, v in dictionary.items()}

input_FUND_NAME = invert_dict(FUND_NAME)

    


In [144]:
#創造FUND物件的格式 之後可以用的到
class FUND:
    def __init__(self, name: str, aum: float, management_fee:float, sector:dict, country:dict, holdings:dict):
        self.name = name
        self.aum = aum
        self.management_fee = management_fee
        self.sector = sector
        self.country = country
        self.holdings = holdings


In [145]:
#read_data
fund_info = {}
# counter = 0
for filename in os.listdir(FUND_info_path):
    isin = filename.split("_")[0]
    # if counter > 5:
    #     break
    if filename.endswith(".json"):
        path = os.path.join(FUND_info_path, filename)
        # counter += 1
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
        fund_info[isin] = data

print(f"讀入了{len(fund_info)}個FUND的基礎資訊")

讀入了54個FUND的基礎資訊


In [146]:
#把想分析的基金打在這就好
target = "JPM Asia Equity High Income Fund"
isin = fund_info[input_FUND_NAME[target]]["fundData"]["shareClass"]["cusip"]


In [147]:
#基金實際大小
AUM = fund_info[isin]["fundData"]["aum"]["value"]
management_fee = fund_info[isin]["fundData"]["shareClass"]["expenses"]["managementFee"]
print(f"基金實際大小: {AUM}", "||", f"管理費: {management_fee}%")


基金實際大小: 581038295.47 || 管理費: 1.5%


In [148]:
#portfolio sector analysis
effective_date = fund_info[isin]["fundData"]["portfolioAnalysisBySector"]["effectiveDate"]
sector = {item["name"]: item["value"] for item in fund_info[isin]["fundData"]["portfolioAnalysisBySector"]["data"]}

# https://plotly.com/python/builtin-colorscales/
# https://plotly.com/python/pie-charts/

sector_name = list(sector.keys())
sector_value = list(sector.values())

sector_fig = px.pie(
    #一定要的parameters
    names = sector_name,
    values = sector_value,
    title = f"{target}產業圓餅圖_{effective_date}",
    #特別可操控的param
    # hole = 0.25,
    color_discrete_sequence = px.colors.sequential.Blues[::-1],
    # # hover_data=['value'], 
    # labels={'value':'sector'}
    # hover_name=['']
)

max_idx = sector_value.index(max(sector_value))
pulls = [0.08 if i == max_idx else 0 for i in range(len(sector))]

sector_fig.update_traces(
    pull = pulls
)

sector_fig.show()




In [150]:
#portfolio country analysis
effective_date = fund_info[isin]["fundData"]["portfolioAnalysisByCountry"]["effectiveDate"]
country = {item["name"]: item["value"] for item in fund_info[isin]["fundData"]["portfolioAnalysisByCountry"]["data"]}

country_name = list(country.keys())
country_value = list(country.values())

country_fig = px.pie(
    #一定要的parameters
    names = country_name,
    values = country_value,
    title = f"{target}國家圓餅圖_{effective_date}",
    #特別可操控的param
    # hole = 0.25,
    color_discrete_sequence = px.colors.sequential.OrRd[::-1],
    # # hover_data=['value'], 
    # labels={'value':'sector'}
    # hover_name=['']
)

max_idx = country_value.index(max(country_value))
pulls = [0.08 if i == max_idx else 0 for i in range(len(country))]

country_fig.update_traces(
    pull = pulls
)

country_fig.show()